In [7]:
#Fuzzy Logic Car Controller - Cruise Control
#Facts: Speed limits, road types, slopes.
import time
import random
import math
speedLimits = [50,60,70,80,90,100]
roadType = []
currentLimit = random.choice(speedLimits)
carSpeed = 0
randomness = 1
deceleration = 0
acceleration = 0.0

def acceleration(limit, speed):
    limitDiff = limit - speed
    if(limitDiff < 60):
        acceleration = math.ceil(limitDiff / 4)
    else:
        acceleration = math.ceil(limitDiff / 6.5)
    return acceleration

class road:
    def __init__(self, roadType, roadDecel):
        self.roadType = roadType
        self.roadDecel = roadDecel
roadType.append(road("City", 5))
roadType.append(road("Suburb", 10))
roadType.append(road("Dirt", 15))
currentRoad = roadType[random.randint(0,2)]
print()
while(True):
    time.sleep(2)
    carSpeed -= currentRoad.roadDecel
    carSpeed += currentRoad.roadDecel
    if(carSpeed < currentLimit):    
        carSpeed += acceleration(currentLimit, carSpeed)
    if(carSpeed > currentLimit):
        carSpeed = currentLimit
    if(random.random() > randomness):
        currentLimit = random.choice(speedLimits)
    print("Speed:")
    print(carSpeed)
    print("Limit:")
    print(currentLimit)
    print("Road Type:")
    print(currentRoad.roadType)
    
    



Speed:
13
Limit:
80
Road Type:
Dirt
Speed:
24
Limit:
80
Road Type:
Dirt
Speed:
38
Limit:
80
Road Type:
Dirt
Speed:
49
Limit:
80
Road Type:
Dirt
Speed:
57
Limit:
80
Road Type:
Dirt
Speed:
63
Limit:
80
Road Type:
Dirt
Speed:
68
Limit:
80
Road Type:
Dirt
Speed:
71
Limit:
80
Road Type:
Dirt
Speed:
74
Limit:
80
Road Type:
Dirt
Speed:
76
Limit:
80
Road Type:
Dirt
Speed:
77
Limit:
80
Road Type:
Dirt
Speed:
78
Limit:
80
Road Type:
Dirt
Speed:
79
Limit:
80
Road Type:
Dirt
Speed:
80
Limit:
80
Road Type:
Dirt
Speed:
80
Limit:
80
Road Type:
Dirt
Speed:
80
Limit:
80
Road Type:
Dirt


KeyboardInterrupt: 

In [6]:
!pip install -U py==1.10